In [8]:
# BUILD YOUR OWN TAXONOMY BASED ON LDA and MANUAL DATA EXPLORATION
topic_taxonomy = {
    "executive leadership CEO bezos":
    {
        "Executive Scandal":  "hack phone mbs bin salman privacy fbi",
        "Executive Compensation": "billionaire forbes paid compensated receives bonus payraise"
    },
    "business company":
    {
        "Business Earnings":    "quarterly earnings profit revenue dividend stock market",
        "Business Promotion": "slashes superbowl advertisement ads discount ",
        "Video":  "series prime season Oscar show ",
        "Products":    "alexa echo bestseller dress clothes retail airpod apple macbook",
        "Business Competition":  "flipkart Disney hbo Netflix showtime "
    },
    "location branch plant store":
    {
        "Facility Opening":    "open expand facility ramp",
        "Facility Relocation": "closure move shutdown relocate"
    },
    "employee worker":
    {
        "Workforce Expansion":  "jobs hiring openings recruitment looking to hire",
        "Workforce Downsizing": "job cut layoff lay off reorganization termination downsizing",
        "Workforce Discontent": "killed concerns dies dissatisfaction labor crew union strike dispute unrest boycott revolt"
    },
    "legal"	:
    {
        "Legal Lawsuit":        "conviction guilty sue battle in court ruling sentencing class action lawsuit litigation settlement appeals trial testify",
        "Legal Investigation":  "law FBI probe allegation indictment investigation face charges grilled over",
        "Legal Scandal":        "scandal revelation whistleblower accusing accusation"
    }
}


In [9]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path = 'C:/Users/subha/Downloads/'

In [10]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [11]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [12]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [13]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [26]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 1 result
def classify_topics(input, vectors):
    feed_score = {} #dict()
    for key, value in topic_taxonomy.items():
        max_value_score = {} #dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:1]

In [15]:
import json

In [16]:
json_data=open("anly610_amazon_deduped_new.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))

In [117]:
data = []
i = 0
for record in feeds_read_from_file:
    dict1 = {}
#     dict['id'] = i
    try:
        dict1['title'] = record['title']
        dict1['topic'] = classify_topics(str(record['title']), model_word2vec)[0][0]
        dict1['max_score'] = classify_topics(str(record['title']), model_word2vec)[0][1]
        i+=1
        data.append(dict1)
    except:
        print('Title not matched with any of the topic, ignored')
#     print(dict1['title'], dict1['output'])
        i+=1

Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored
Title not matched with any of the topic, ignored


In [118]:
import itertools
from operator import itemgetter

In [119]:
summary1 = sorted(data, key=itemgetter('topic', 'max_score'), reverse=True)

In [120]:
for key, value in itertools.groupby(summary1, key=itemgetter('topic')):
    print (key)
    x = 0
    for i in value:
        x+=1
        print ("      ", i.get('title'))
        if x==10:
            break

Workforce Expansion
       Amazon looking for staff after new centre creates 600 jobs
       Facebook set to hire 1,000 staff in post-Brexit London | News
       Pete Buttigieg's campaign used a company criticized for its low wages to hire workers, even after Buttigieg has been critical of the gig economy (AMZN)
       Pete Buttigieg's campaign used a company criticized for its low wages to hire workers, even after Buttigieg has been critical of the gig economy (AMZN)
       Pete Buttigieg's campaign used a company criticized for its low wages to hire workers, even after Buttigieg has been critical of the gig economy
       Pete Buttigieg's campaign used a company criticized for its low wages to hire workers, even after Buttigieg has been critical of the gig economy
       Amazon opens new corporate office in Manchester creating 600 jobs
       Amazon opens new corporate office in Manchester creating 600 jobs
       Amazon opens new corporate office in Manchester creating 600 jobs
    